# business 跑模型数据生成

In [ ]:
import pandas as pd
import numpy as np
import json
import os

In [ ]:
# 数据格式的转换

def protect_excel_formula(df):
    for col in df.columns:
        df[col] = df[col].apply(
            lambda x: f"'{x}" if isinstance(x, str) and x.startswith(('=', '+', '-', '@')) else x
        )
    return df


def json_to_excel(json_path,excel_path):
    """
    json_path:input read file json path
    excel_path:output write file excel path
    return:None
    """
    data_excel = []
    with open(json_path, 'r', encoding='utf-8') as json_file:
        all = json_file.readlines()
        header = list(json.loads(all[0]).keys())
        for line in all:
            data =  list(json.loads(line).values())
            data_excel.append(data)
    df = pd.DataFrame(data_excel,columns=header)
    dfs = protect_excel_formula(df)
    dfs.to_excel(excel_path,index=False)
    print("{} data save success".format(json_path))


def json_to_csv(json_path,excel_path):
    """
    json_path:input read file json path
    excel_path:output write file excel path
    return:None
    """
    data_excel = []
    with open(json_path, 'r', encoding='utf-8') as json_file:
        all = json_file.readlines()
        header = list(json.loads(all[0]).keys())
        for line in all:
            data =  list(json.loads(line).values())
            data_excel.append(data)
    df = pd.DataFrame(data_excel,columns=header)
    dfs = protect_excel_formula(df)
    dfs.to_csv(excel_path,index=False)
    print("{} data save success".format(json_path))



file_names = os.listdir('./data/json')
for file_name in file_names[3:]:
    print(file_name)
    try:
        input_path = './data/json/'+file_name
        ouput_put = './data/excel/'+file_name.replace("json",'xlsx')
        json_to_excel(input_path,ouput_put)
    except:
        ouput_put = './data/excel/'+file_name.replace("json",'csv')
        json_to_csv(input_path,ouput_put)
# 参数1 输入路径json 参数2 输出路径
# json_to_csv("***.json","***.csv")

In [ ]:
# -----------------------------------------------tips-----------------------------------------------
# tips data ==> generate summary data  对tips数据集进行分组统计，分组条件business_id
# df = pd.read_csv('./data/excel/yelp_academic_dataset_tip.csv')
# result = df.groupby('business_id').agg({'user_id':len,
#                         'compliment_count': [np.average,np.std],
#                         }).reset_index().values.tolist()
# pd.DataFrame(result, columns = ['business_id','tips_count','compliment_avg','compliment_std'
#                                 ]).to_csv('./data/excel/tips_result.csv', index=False)

In [ ]:
# -----------------------------------------------review-----------------------------------------------
# from nltk.sentiment import SentimentIntensityAnalyzer

# # content_score 计算评论情感的分数
# def analyze_sentiment(sentence):
#     sid = SentimentIntensityAnalyzer()
#     sentiment_scores = sid.polarity_scores(sentence)
#     return sentiment_scores['compound']

# 遍历数据集并保存评分结果
# # calculate review text score
# # df = pd.read_csv('./data/excel/yelp_academic_dataset_review.csv')
# df = pd.read_csv('review_new.csv')
# result = []
# for index,row in df.iterrows():
#     print(index,analyze_sentiment(row['text']))
#     result.append([index,analyze_sentiment(row['text'])])
# pd.DataFrame(result,columns=['index','score']).to_csv('./data/excel/review_score_result_1_new.csv',index=False)

# # 合并 review与情感评分数据
df = pd.read_csv('./data/excel/yelp_academic_dataset_review.csv')
df = df[['user_id', 'business_id', 'stars', 'useful', 'funny','cool']]
dfs = pd.read_csv('./data/excel/review_score_result_1.csv')
df = pd.concat([df,dfs],axis=1)
df = df[['user_id', 'business_id', 'stars', 'useful', 'funny','cool','score']]
df.to_csv('./data/excel/review_merge.csv',index=False)


# 对合并好的数据进行分组summary 统计
df = pd.read_csv('./data/excel/review_merge.csv')
result = df.groupby('business_id').agg({'user_id':pd.Series.nunique,
                        'stars': [np.average, np.std],
                        'score': [np.average], 
                        'useful': [np.average], 
                        'funny': [np.average],
                        'cool': [np.average], 
                        }).reset_index().values.tolist()
pd.DataFrame(result, columns = ['business_id','user_count','star_avg','star_std','contents_score_avg',
                                'useful_avg','funny_avg','cool_avg',]).to_csv('./data/excel/review_result.csv', index=False)

In [ ]:
# -----------------------------------------------photo-----------------------------------------------
# 基于图片计算 memory_score h s v
import cv2
import os
from resmem import ResMem, transformer
from PIL import Image
import os
import pandas as pd
import numpy as np

# memory_score
model = ResMem(pretrained=True)
def pic_score_pre(pic_path):
    img = Image.open(pic_path) # This loads your image into memory
    img = img.convert('RGB') 
    # This will convert your image into RGB, for instance if it's a PNG (RGBA) or if it's black and white.
    model.eval()
    # Set the model to inference mode.
    image_x = transformer(img)
    # Run the preprocessing function
    # print(image_x.shape)
    prediction = model(image_x.view(-1, 3, 227, 227))
    # For a single image, the image must be reshaped into a batch
    # with size 1.
    # Get your prediction!
    return round(prediction[0][0].item(),3)



def get_hsv(pic_path):
    # read pic
    img = cv2.imread(pic_path)

    # transform hsv color
    hsv_img = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)

    # get h s v 
    h,s,v = cv2.split(hsv_img)

    # print(h, s, v)
    # print(np.mean(h),np.mean(s),np.mean(v))

    return round(np.mean(h),3),round(np.mean(s),3),round(np.mean(v),3)

# def pic_detail_to_summary(detail_file,output_path):
#     file = pd.read_excel(detail_file)


fs = open('./data/log/pic.log','w+',encoding='utf-8')

# photo就是原数据，yelp上下载的
pic_df = pd.read_excel('./data/excel/photos.xlsx')
# print(pic_df)

pic_df['memory_score'] = 0
pic_df['h'] = 0
pic_df['s'] = 0
pic_df['v'] = 0

for index,row in pic_df.iterrows():
    # print(index,row['photo_id'],row['business_id'],row['caption'],row['label'])
    pic_path = './data/pic/'+row['photo_id']+'.jpg'
    print(pic_path)
    try:
        pic_score = pic_score_pre(pic_path)
        h,s,v = get_hsv(pic_path)
        pic_df.iloc[index,-4] = pic_score
        pic_df.iloc[index,-3] = h
        pic_df.iloc[index,-2] = s
        pic_df.iloc[index,-1] = v
        print('    '.join([row['photo_id'],str(pic_score),str(h),str(s),str(v)]),file=fs)
        print('    '.join([row['photo_id'],str(pic_score),str(h),str(s),str(v)]))
    except:
        print(row['photo_id'],file=fs)
        # pass

pic_df.to_excel('./data/excel/photos_detail.xlsx',index=False) 
# photo_detail 就是 photo_result


# # pic_df= pic_df.head(5)

# col_name = ['business_id','pic_count','pic_type_count','pic_type_content']
# for col in ['memory_score','h','s','v']:
#     for label in ['inside','outside','menu','drink','food']:
#         for calculate in ['avg','max']:
#             col_name.append(col+'_'+label+'_'+calculate)

# all_calculate = []
# for business_id in  pic_df['business_id'].unique():
#     line = []
#     pic_count = len(pic_df[pic_df['business_id']==business_id]['photo_id'].unique())
#     pic_type_count = len(pic_df[pic_df['business_id']==business_id]['label'].unique())
#     pic_type_content = pic_df[pic_df['business_id']==business_id]['label'].unique()
#     # print(business_id,pic_count,pic_type_count,pic_type_content)
#     line.append(business_id)
#     line.append(pic_count)
#     line.append(pic_type_count)
#     line.append(pic_type_content)
        
#     for col in ['memory_score','h','s','v']:
#         for label in ['inside','outside','menu','drink','food']:
#             if len(pic_df[(pic_df['business_id']==business_id)&(pic_df['label']==label)][col])==0:
#                 avg_v = 0
#                 max_v = 0
#             else:
#                 avg_v = np.sum(pic_df[(pic_df['business_id']==business_id)&(pic_df['label']==label)][col])/len(pic_df[(pic_df['business_id']==business_id)&(pic_df['label']==label)][col])
#                 max_v = np.max(pic_df[(pic_df['business_id']==business_id)&(pic_df['label']==label)][col])
#             # print(col,label,avg_v,max_v)
#             line.append(avg_v)
#             line.append(max_v)
#     all_calculate.append(line)

# pic_df_summary = pd.DataFrame(all_calculate,columns=col_name)

# pic_df_summary.to_excel('./data/excel/photos_summary.xlsx',index=False)


In [ ]:
# -----------------------------------business-----------------------------------------------
# 通过 上面的函数将json 转换为 xlsx
# 提取部分变量  business_id	stars	review_count	is_open	categories_counts   category_counts  是根据 category进行计数统计
business = pd.read_excel("data/excel/yelp_academic_dataset_business.xlsx")
business = business[['business_id','stars','review_count','is_open','categories','user_count']]
business['categories_counts'] = business['categories'].str.split(',').str.len()
business.to_excel("./data/excel/business_result.xlsx",index=False)

In [43]:
def protect_excel_formula(df):
    for col in df.columns:
        df[col] = df[col].apply(
            lambda x: f"'{x}" if isinstance(x, str) and x.startswith(('=', '+', '-', '@')) else x
        )
    return df

In [ ]:
# 数据的合并

import pandas as pd

business = pd.read_excel('./data/excel/business_result.xlsx')
review = pd.read_csv('./data/excel/review_result.csv')
# tips = pd.read_csv('./data/excel/tips_result.csv')
# photos_detail 等于 photos_result
# photos = pd.read_excel("./data/excel/photos_result.xlsx")
# 数据格式处理
business = protect_excel_formula(business)
df1 = pd.merge(business,review,left_on='business_id',right_on='business_id',how='left')

# # 过滤掉name没有识别出来的数据
# result = df1.dropna()

df1.to_csv('./data/excel/business_feature.csv',index=False)


In [57]:
print(len(df1),df1.columns)

150346 Index(['business_id', 'stars', 'review_count', 'is_open', 'categories_counts',
       'user_count', 'star_avg', 'star_std', 'contents_score_avg',
       'useful_avg', 'funny_avg', 'cool_avg'],
      dtype='object')


In [58]:
temp = pd.read_csv('./data/excel/yelp_academic_dataset_review.csv')
len(temp)

6990280